# RAG and Python Function Deployment

In this notebook, we create and deploy a Python function that processes a list of clauses/fragments from an NDA and generates corrections and suggestions using the Retrieval-Augmented Generation (RAG) process. The function performs the following steps:

1. **Create the Chroma Knowledge Database (KDB)**:
    - Utilize the file 'NDA Standards.txt' to construct the KDB of predefined standards.

2. **Process Each Clause/Fragment**:
    - For each clause/fragment in the input list, the function:
        - Retrieves the top 3 most similar standards from the KDB using vector search.
        - Includes the clause/fragment and the retrieved standards into a prompt template for the Large Language Model (LLM).

3. **Run LLM Inference**:
    - Execute LLM inference on watsonx.ai to generate an answer for each clause/fragment.

4. **Return Results**:
    - Outputs a list of results containing the corrections and suggestions for each input clause/fragment.


In [4]:
# Please enter your credentials needed to start the 'APIclient'.

credentials = {
    "url": ...,
    "apikey": ...
}

In [26]:
import os
project_id = os.environ["PROJECT_ID"]

In [27]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)
# Please create a watsonx.ai deployment space, and update the next parameter with the Space UID:
space_uid = ...
client.set.default_space(space_uid)

'SUCCESS'

### Promote Necessary Assets to the Deployment Space.

The following assets are promoted to the deployment space to be used in the deployed function
* The file 'NDA Standards.txt' with the NDA standards
* LLM Prompt template & instructions

In [28]:
promote_files_to_space=['NDA Standards.txt', 'RAG template - LLAMA 3']
space_asset_dict={}
for file in project.get_assets():
    if file['name'] in promote_files_to_space:
        space_asset_dict[file['name']] = client.spaces.promote(file['asset_id'], project_id, space_uid)
    
space_asset_dict

{'RAG template - LLAMA 3': 'a7f512f6-04d4-4ad0-b392-176541f69f2b',
 'NDA Standards.txt': 'b097c2f8-24e7-4b39-956a-2b2c44f7ca53'}

In [29]:
params = {'project_id': os.environ["PROJECT_ID"], 'space_uid': space_uid, "credentials": credentials, "template_name": "RAG template - LLAMA 3"}

In [30]:
def rag_scoring_pipeline_function(params=params, space_asset_dict=space_asset_dict):
    import subprocess
    subprocess.check_output("pip install --upgrade sentence_transformers --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install --upgrade ibm-watsonx-ai==0.1.7 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install langchain==0.0.354 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install --upgrade langchain-community --user", stderr=subprocess.STDOUT, shell=True) 
    subprocess.check_output("pip install --upgrade langchain-core --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install pydantic==1.10.11 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install langchain_ibm --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install chromadb==0.3.26 --user", stderr=subprocess.STDOUT, shell=True)

    
    
    credentials = params["credentials"]
    project_id  = params["project_id"]
    
    # Set the WML Client
    from ibm_watsonx_ai import APIClient
    try:
        client = APIClient(credentials)
        space_uid = params["space_uid"]
        client.set.default_space(space_uid)
    except Exception as e:
        print(f"Error initializing WML client: {str(e)}")
        raise
        
        
    # Construct Knowledge base
    try: 
        filename = 'NDA Standards.txt'
        path = client.data_assets.download(space_asset_dict[filename], filename)
        with open(path, 'r') as f:
            content = f.read()
            
        from langchain.docstore.document import Document

        standards = []
        for line in content.strip().split('\n'):
            if line.strip():
                line_split = line.split(':', 1)
                standard_number = int(line_split[0][-1])
                text = line_split[1].strip()
                standards.append({"page_content": text, "metadata": {"standard_number": standard_number}})

        texts = [Document(**standard) for standard in standards]
            
        from langchain_ibm import WatsonxEmbeddings
        from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
        from langchain.vectorstores import Chroma

        embeddings = WatsonxEmbeddings(
            model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
            url=credentials["url"],
            apikey=credentials["apikey"],
            project_id=project_id
            )

        docsearch = Chroma.from_documents(texts, embeddings)
        
        
    except Exception as e:
        print(f"Error initializing the knowledge base: {str(e)}")
        raise
    
    
    # Gen AI via WatsonX.AI
    try:
        from ibm_watsonx_ai.foundation_models.prompts import PromptTemplateManager
        from ibm_watsonx_ai.foundation_models.utils.enums import PromptTemplateFormats
        from ibm_watson_machine_learning.foundation_models import Model
        
        prompt_mgr = PromptTemplateManager(credentials=credentials,
                                           space_id=space_uid)
        df = prompt_mgr.list()
        prompt_id = space_asset_dict['RAG template - LLAMA 3']
        prompt_text = prompt_mgr.load_prompt(prompt_id=prompt_id, astype=PromptTemplateFormats.STRING)
        prompt_model = prompt_mgr.load_prompt(prompt_id)
        model = Model(
            model_id=prompt_model.model_id,
            params=prompt_model.model_params,
            credentials=credentials,
            project_id=project_id
        )
        
    except Exception as e:
            
        print(f"Error creating LLM: {str(e)}")
        
    # Helper function for generating response
    from typing import Tuple

    def generate( context:Tuple[str, str], model_in = model, prompt_text = prompt_text ):

        filled_prompt_text = prompt_text.format(standards=context[0], fragment=context[1])

#         print(filled_prompt_text)

        generated_response = model_in.generate( filled_prompt_text )

        if ( "results" in generated_response ) \
           and ( len( generated_response["results"] ) > 0 ) \
           and ( "generated_text" in generated_response["results"][0] ):
            return generated_response["results"][0]["generated_text"]
        else:
            print( "The model failed to generate an answer" )
            print( "\nDebug info:\n" + json.dumps( generated_response, indent=3 ) )
            return ""
        
    def score(payload):
        # payload scheme:
        #   Run RAG pipeline with pre-filled "feedback" in log:
        #     { fields: [<anything>], values: [[frag1, ..., fragN]] }
        
        fields = {payload['input_data'][0]['fields'][i]: payload['input_data'][0]['values'][i] for i in range(min(len(payload['input_data'][0]['fields']), len(payload['input_data'][0]['values'][0])))}
        frags  = payload['input_data'][0]['values'][0]
        
#         print(fields, frags)

        try:
            responses = []
            for frag in frags:
                # Step 1: Get the 3 most similar clauses from the knowledge base
                similar_docs = docsearch.similarity_search(frag, k=3)

                # Combine the similar clauses into a single string with the correct format
                combined_standards = "\n".join(
                    [f"Predefined standard {doc.metadata['standard_number']}: {doc.page_content}" for doc in similar_docs]
                )

                # Step 2: Pass the standards and the input fragment to the model
                response = generate((combined_standards, frag))

                # Step 3: Print the response
                responses.append(response)
                
        except Exception as e:
            
            print(f"Error while getting LLM response: {str(e)}")
            raise
        
        scoring_response = {
            "predictions": [
                {
                    "fields": [
                        "llm_prediction"
                    ],
                    "values": [responses]
                }]}
        
        return scoring_response

    
    return score


In [32]:
values = ["The receiving party agrees to keep the confidential information disclosed to them confidential for a period of eighteen months following the termination of this agreement.",
         """Governance
This Agreement shall be governed by and construed in accordance with the European law without regard to its choice of law principles. Any disputes that relate to the execution, interpretation, construction, performance, or enforcement of the Agreement will be brought and resolved solely and exclusively in the courts of Brussels, Belgium."""]

In [33]:
rag_scoring_pipeline_function()({"input_data": [{"fields": ["Text"],"values": [values]}]})

Successfully saved data asset content to file: 'NDA Standards.txt'


{'predictions': [{'fields': ['llm_prediction'],
   'values': [['**1. Compliance Issues:** \nPredefined standard 1: The fragment violates this standard as it requires confidentiality for a period of eighteen months, which exceeds the maximum period of one year specified in the standard.\n\n**2. Corrections:** \nThe receiving party agrees to keep the confidential information disclosed to them confidential for a period of [twelve months] following the termination of this agreement.',
     '**1. Compliance Issues:**\nPredefined standard 2: The governing law of the agreement should always be Polish and not international or based on the laws of any other country.\n\n**2. Corrections:**\n"Governance\nThis Agreement shall be governed by and construed in accordance with the [Polish] law without regard to its choice of law principles. Any disputes that relate to the execution, interpretation, construction, performance, or enforcement of the Agreement will be brought and resolved solely and exclu

### Deploying it

In [34]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)
space_uid = ...
client.set.default_space(space_uid)

pyfunc_swspec_id = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'rag_function_watsonx_ai',
    client.repository.FunctionMetaNames.DESCRIPTION: 'Watsonx_Function_for_RAG_with_Chroma',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}

watsonx_function_details = client.repository.store_function(meta_props=meta_data, function=rag_scoring_pipeline_function)

In [35]:
import uuid

watsonx_function_uid = client.repository.get_function_id(watsonx_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "rag-function-watsonx-ai-deployment",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "Wrapper function to perform RAG",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.SERVING_NAME: "rag_with_wx_es" + str(uuid.uuid1()).split('-')[0]
}

watsonx_deployment_details = client.deployments.create(watsonx_function_uid, meta_props=meta_props)



######################################################################################

Synchronous deployment creation for id: '249a419b-ffbd-4389-aac1-bcdc9d84689c' started

######################################################################################


initializing..................
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='cd5e103c-75ab-4aac-8c19-c67e9fc1d60e'
-----------------------------------------------------------------------------------------------




### Test the scoring function

In [36]:
watsonx_deployment_id = client.deployments.get_id(watsonx_deployment_details)
values = ["The receiving party agrees to keep the confidential information disclosed to them confidential for a period of eighteen \
months following the termination of this agreement.",
         """Governance
This Agreement shall be governed by and construed in accordance with the European law without regard to its choice of law principles. \
Any disputes that relate to the execution, interpretation, construction, performance, or enforcement of the Agreement will be brought \
and resolved solely and exclusively in the courts of Brussels, Belgium."""]


payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["Text"],
        "values": [values]
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

{'predictions': [{'fields': ['llm_prediction'],
   'values': [['**1. Compliance Issues:** \nPredefined standard 1: The fragment violates this standard as it requires confidentiality for a period of eighteen months, which exceeds the maximum period of one year specified in the standard.\n\n**2. Corrections:** \nThe receiving party agrees to keep the confidential information disclosed to them confidential for a period of [twelve months] following the termination of this agreement.',
     '**1. Compliance Issues:**\nPredefined standard 2: The governing law of the agreement should always be Polish and not international or based on the laws of any other country.\n\n**2. Corrections:**\n"Governance\nThis Agreement shall be governed by and construed in accordance with the [Polish] law without regard to its choice of law principles. Any disputes that relate to the execution, interpretation, construction, performance, or enforcement of the Agreement will be brought and resolved solely and exclu

In [39]:
values = ["""Nondisclosure Agreement
This Nondisclosure Agreement (the "Agreement") is entered into by and between Tech Innovators Polska Sp. z o.o. with its principal offices at ul. Innowacyjna 123, 00-001 Warszawa, Poland ("Disclosing Party") and Creative Solutions Sp. z o.o., located at ul. Przedsiębiorcza 456, 02-002 Kraków, Poland ("Receiving Party") for the purpose of preventing the unauthorized use and disclosure of Confidential Information as defined below.

The parties agree to enter into a confidential relationship with respect to the use and disclosure of certain proprietary and confidential information ("Confidential Information").
""",
         """Definition of Confidential Information
For purposes of this Agreement, "Confidential Information" shall include any information, material, data, or know-how, including trade secrets and proprietary information, that is not generally known to the public and that is disclosed, either written or orally, to be or appears to a reasonable person to be proprietary or confidential. Confidential information in written and oral materials should be identified solely based on the list of confidential materials prepared by a group experts selected from both parites.
""",
         """Exclusions from Confidential Information
Receiving Party's obligations under this Agreement do not extend to information that is: (a) publicly known at the time of disclosure or subsequently becomes publicly known through no fault of Receiving Party; (b) discovered or created by Receiving Party before disclosure by Disclosing Party; (c) learned by Receiving Party through legitimate means other than from Disclosing Party or Disclosing Party's representatives; (d) is disclosed by Receiving Party with Disclosing Party's prior written approval; or (e) is disclosed as required or ordered by a court, administrative agency, or other governmental body.
""",
         """Obligations of Receiving Party
Receiving Party shall hold and maintain the Confidential Information in strictest confidence for the sole and exclusive benefit of Disclosing Party. Receiving Party shall carefully restrict access to Confidential Information to employees as is reasonably required and shall require them to sign nondisclosure restrictions at least as protective as those in this Agreement. Receiving Party shall not, without prior written approval of Disclosing Party, use for Receiving Party's own benefit, publish, copy, or otherwise disclose to others, or permit the use by others for their benefit or to the detriment of Disclosing Party, any Confidential Information. Upon written request by Disclosing Party, Receiving Party shall immediately return to Disclosing Party any and all records, notes, and other written, printed, or tangible materials in its possession pertaining to Confidential Information.
""",
         """Rights in Confidential Information
Disclosing Party shall hold and maintain all rights, title, and interest in and to any Confidential Information. This Agreement and the disclosure of any Confidential Information by Disclosing Party to Receiving Party shall not be construed as granting Receiving Party any rights, title, or interest in the Confidential Information, including any rights in copyright, trademark, patent, or any other intellectual property right.
""",
         """Term
The nondisclosure provisions of this Agreement shall survive the termination of this Agreement and Receiving Party's duty to hold Confidential Information in confidence shall remain in effect until the Confidential Information no longer qualifies as a trade secret or confidential or until Disclosing Party sends Receiving Party written notice releasing Receiving Party from this Agreement, whichever occurs first.
""",
         """Governance
This Agreement shall be governed by and construed in accordance with the European law without regard to its choice of law principles. Any disputes that relate to the execution, interpretation, construction, performance, or enforcement of the Agreement will be brought and resolved solely and exclusively in the courts of Brussels, Belgium.
""",
         """Relationship
Nothing contained in this Agreement shall be deemed to constitute either party a partner, joint venturer, or employee of the other party for any purpose.

Successor and Assigns
This Agreement binds and benefits the heirs, successors, and assignees of the parties.

Severability
If a court finds any provision of this Agreement invalid or unenforceable, the remainder of this Agreement shall be interpreted so as best to effect the intent of the parties.

Waiver
The failure by either party to exercise any right provided in this Agreement shall not be a waiver of prior or subsequent rights.
""",
          """Entire Agreement
This Agreement expresses the complete understanding of the parties with respect to the subject matter and supersedes all prior proposals, agreements, representations, and understandings. This Agreement may not be amended except in a written agreement signed by both parties.

This Agreement and each party's obligations are binding only for direct representatives of the parties. Each party has signed this Agreement through its authorized representative.
"""
         ]

payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["Text"],
        "values": [values]
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

{'predictions': [{'fields': ['llm_prediction'],
   'values': [['**1. Compliance Issues:** \nNone of the predefined standards are explicitly violated in the provided fragment.\n\n**2. Corrections:** \nNo violations were found. The fragment is compliant with every standard.',
     '**1. Compliance Issues:**\n\n* Predefined standard 5: The fragment does not explicitly state that the Disclosing Party shall label or stamp written confidential materials with the word "Confidential" or a similar warning.\n* Predefined standard 6: The fragment does not mention providing written notice for orally transmitted confidential information.\n\n**2. Corrections:**\n\nHere is the corrected fragment:\n\n"Definition of Confidential Information\nFor purposes of this Agreement, "Confidential Information" shall include any information, material, data, or know-how, including trade secrets and proprietary information, that is not generally known to the public and that is disclosed, either written or orally, to